This notebook conatins code to implement DeepFM model, which takes input of BERT embeddings and Word2Vec embeddings to draw out a comparison as to how the recommendation is affected in case of with and without embeddings.

# DeepFM Model to predict our ratings


> With and Without Embeddings

We chose DeepFM model for our recommendation task as it is able to capture complex relationships between user and item interactions.

We are giving two types of inputs to it:
1. With embeddings (BERT and Word2Vec respectively)
2. No embeddings



### **Installing libraray to use the pretrained dataset for our prediction of ratings task**

In [ ]:
!pip install -U LibRecommender

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
import matplotlib.pyplot as plt
from libreco.data import split_by_ratio, DatasetFeat, DataInfo, DatasetPure
from libreco.algorithms import FM, WideDeep, DeepFM, AutoInt, DIN
from libreco.evaluation import evaluate

# remove unnecessary tensorflow logging
import os
import tensorflow as tf

Instructions for updating:
non-resource variables are not supported in the long term


# **Giving the embeddings input to DeepFM first**

In [ ]:
ratings_embeds = ratings_embeds.rename(columns={"user_id": "user", "movie_inner_id": "item", "rating": "label"})

In [ ]:
ratings_embeds.head()

,user,item,label,0,1,2,3,4,5,6,...,290,291,292,293,294,295,296,297,298,299
0,1,600.0,3,-1.721478,-0.832000,0.617502,0.018194,0.058983,0.648678,-0.593404,...,-0.025236,0.020541,-0.036698,0.004546,0.007758,-0.031365,-0.026731,-0.020137,0.040260,0.020498
1,1,814.0,3,-1.241369,-0.358426,0.669684,0.563950,-0.039014,0.053344,0.013220,...,-0.046992,0.048602,-0.047589,-0.021901,0.017764,-0.061331,-0.008716,0.063710,-0.005912,0.032193
2,1,1126.0,5,2.869924,-0.349474,0.367362,-0.948937,0.147643,-0.270624,-0.341831,...,0.062469,0.021386,-0.046009,0.023518,-0.029457,-0.013069,-0.006324,-0.027765,-0.026619,-0.004149
3,1,818.0,4,-0.985229,0.629930,0.268292,-0.406528,-0.052441,0.172558,0.295975,...,0.055384,0.012652,0.010170,0.021458,-0.024664,0.035982,0.007667,0.017644,-0.004264,-0.007093
4,1,542.0,5,-1.700459,-0.603799,0.513942,-0.038975,0.022973,-0.035123,-0.107125,...,-0.048954,0.118035,-0.018279,-0.031425,-0.026168,-0.019687,0.130536,-0.014735,-0.015230,0.012594


# **Splitting the data into train, test and validation sets**

In [ ]:
train_data, test_data, eval_data = split_by_ratio(ratings_embeds, multi_ratios=[0.8, 0.1, 0.1], seed=42)

In [ ]:
train_data['user']

0            1
1            1
2            1
3            1
4            1
          ... 
367721    6040
367722    6040
367723    6040
367724    6040
367725    6040
Name: user, Length: 294284, dtype: int64

In [ ]:
train_data.columns = ratings_embeds.columns

In [ ]:
emb_cols = ratings_embeds.columns[-300:]
dense_cols = list(emb_cols)

In [ ]:
# specify complete columns information
sparse_col = []
dense_col = dense_cols
user_col = []
item_col = dense_cols


In [ ]:
train_data

,user,item,label,0,1,2,3,4,5,6,...,290,291,292,293,294,295,296,297,298,299
0,1,600.0,3,-1.721478,-0.832000,0.617502,0.018194,0.058983,0.648678,-0.593404,...,-0.025236,0.020541,-0.036698,0.004546,0.007758,-0.031365,-0.026731,-0.020137,0.040260,0.020498
1,1,814.0,3,-1.241369,-0.358426,0.669684,0.563950,-0.039014,0.053344,0.013220,...,-0.046992,0.048602,-0.047589,-0.021901,0.017764,-0.061331,-0.008716,0.063710,-0.005912,0.032193
2,1,1126.0,5,2.869924,-0.349474,0.367362,-0.948937,0.147643,-0.270624,-0.341831,...,0.062469,0.021386,-0.046009,0.023518,-0.029457,-0.013069,-0.006324,-0.027765,-0.026619,-0.004149
3,1,818.0,4,-0.985229,0.629930,0.268292,-0.406528,-0.052441,0.172558,0.295975,...,0.055384,0.012652,0.010170,0.021458,-0.024664,0.035982,0.007667,0.017644,-0.004264,-0.007093
4,1,542.0,5,-1.700459,-0.603799,0.513942,-0.038975,0.022973,-0.035123,-0.107125,...,-0.048954,0.118035,-0.018279,-0.031425,-0.026168,-0.019687,0.130536,-0.014735,-0.015230,0.012594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367721,6040,402.0,4,-0.608836,-0.006751,-0.387712,-0.128381,0.073968,0.363810,0.152113,...,0.021822,-0.031334,0.017498,-0.006875,-0.030916,0.034221,0.022270,0.010347,-0.015040,0.003375
367722,6040,404.0,4,-0.215605,-0.161121,0.002349,0.689052,-0.528751,0.772533,0.451087,...,0.023696,-0.002356,0.009762,0.022879,0.020742,-0.038606,0.007950,0.020253,-0.014205,0.011744
367723,6040,413.0,4,0.451951,-1.071415,0.740344,-0.185958,0.381112,0.418187,-0.060973,...,-0.039821,-0.032852,0.028733,0.016534,0.059977,-0.002145,0.000448,0.035749,0.020471,-0.001455
367724,6040,1137.0,5,1.238230,0.133431,-0.557367,-0.081861,-0.378480,0.059226,-0.188412,...,0.018659,0.052624,0.039146,-0.030402,-0.004675,0.049967,-0.022842,-0.008974,-0.009273,-0.040527


**Here, we are training our model by leveraging pre-trained dataset called DatasetFeat provided by LibRecommender**

In [ ]:
train_data_to_model, data_info = DatasetFeat.build_trainset(train_data, user_col, item_col, sparse_col, dense_col)
eval_data_to_model = DatasetFeat.build_evalset(eval_data)
test_data_to_model = DatasetFeat.build_testset(test_data)

In [ ]:
tf.compat.v1.reset_default_graph()

# **Initialising model parameters**

In [ ]:
deepfm = DeepFM("rating", data_info, embed_size=16, n_epochs=10,
                    lr=1e-5, lr_decay=False, reg=0.001, batch_size=5000,
                   use_bn=True, dropout_rate=0.4,
                    hidden_units=[256,128,64,32], loss_type="mae")

## **Model training - with embeddings input**

In [ ]:
deepfm.fit(
    train_data_to_model,
    neg_sampling=False,
    verbose=2,
    shuffle=True,
    eval_data=eval_data_to_model,
    metrics=["loss", "rmse"],
)

/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
Instructions for updating:
Colocations handled automatically by placer.


Training start time: 2024-03-13 01:19:56


/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:47: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  net = tf.layers.dropout(net, dropout_rate, training=is_training)


total params: 1,416,275 | embedding params: 125,364 | network params: 1,290,911


train: 100%|██████████| 59/59 [03:05<00:00,  3.14s/it]


...

	 eval rmse: 3.0187
	 eval rmse: 3.0187


train: 100%|██████████| 59/59 [02:21<00:00,  2.40s/it]


Epoch 10 elapsed: 141.849s
	 train_loss: 15.6138


eval_pointwise: 100%|██████████| 5/5 [00:04<00:00,  1.11it/s]


	 eval rmse: 3.0187
	 eval rmse: 3.0187


# **RESULT WITH EMBEDDINGS**

In [ ]:
result_embeds = evaluate(model=deepfm, data=test_data_to_model, neg_sampling='True', metrics=["loss", "rmse", "mae", "r2"])
print("evaluate_result: ", result_embeds)

eval_pointwise: 100%|██████████| 9/9 [00:12<00:00,  1.40s/it]

evaluate_result:  {'loss': 0.70710677, 'rmse': 0.70710677, 'mae': 0.5, 'r2': -1.0}


In [ ]:
result_reviews = evaluate(model=deepfm, data=test_data, neg_sampling='True', metrics=["loss", "rmse", "mae", "r2"])
print("evaluate_result: ", result_reviews)

eval_pointwise: 100%|██████████| 9/9 [00:11<00:00,  1.28s/it]

evaluate_result:  {'loss': 0.70710677, 'rmse': 0.70710677, 'mae': 0.5, 'r2': -1.0}


In [ ]:
ratings = ratings.rename(columns={"user_id": "user", "movie_id": "item", "rating": "label"})

In [ ]:
ratings

,user,item,label
0,1,1193,5
1,1,661,3
2,1,914,3
4,1,2355,5
6,1,1287,5
...,...,...,...
1000203,6040,1090,3
1000204,6040,1091,1
1000206,6040,562,5
1000207,6040,1096,4


## **CONSIDERING THE DATA WITHOUT EMBEDDINGS**

In [ ]:
train_data1 = train_data.drop(columns=dense_col)
eval_data1 = eval_data.drop(columns=dense_col)
test_data1 = test_data.drop(columns=dense_col)


In [ ]:
ratings.head()

,user,item,label
0,1,1193,5
1,1,661,3
2,1,914,3
4,1,2355,5
6,1,1287,5


In [ ]:
train_data1, test_data1, eval_data1 = split_by_ratio(ratings, multi_ratios=[0.8, 0.1, 0.1])

In [ ]:
train_data1

,user,item,label
0,1,1193,5
1,1,661,3
2,1,914,3
4,1,2355,5
6,1,1287,5
...,...,...,...
1000139,6040,1392,5
1000140,6040,1394,3
1000141,6040,866,4
1000142,6040,2348,2


In [ ]:
train_data1['user']

0             1
1             1
2             1
4             1
6             1
           ... 
1000139    6040
1000140    6040
1000141    6040
1000142    6040
1000143    6040
Name: user, Length: 738199, dtype: int64

In [ ]:
train_data1.columns = ratings.columns

In [ ]:
emb_cols = ratings.columns[-300:]
dense_cols = list(emb_cols)

In [ ]:
# specify complete columns information
sparse_col = []
dense_col = []
user_col = []
item_col = []


In [ ]:
train_data1

,user,item,label
0,1,1193,5
1,1,661,3
2,1,914,3
4,1,2355,5
6,1,1287,5
...,...,...,...
1000139,6040,1392,5
1000140,6040,1394,3
1000141,6040,866,4
1000142,6040,2348,2


In [ ]:
train_data1to, data_info1 = DatasetPure.build_trainset(train_data1)
eval_data1to = DatasetPure.build_testset(eval_data1)
test_data1to = DatasetPure.build_testset(test_data1)

In [ ]:
data_info1, len(train_data1to)

(n_users: 6040, n_items: 3001, data density: 4.0726 %, 738199)

In [ ]:
tf.compat.v1.reset_default_graph()

In [ ]:
train_data1.isnull().sum()

user     0
item     0
label    0
dtype: int64

In [ ]:
data_info1

n_users: 6040, n_items: 3001, data density: 4.0726 %

### **NON - EMBEDDING INPUT TO DEEPFM**

In [ ]:
deepfm1 = DeepFM("rating", data_info1, embed_size=5, n_epochs=12,
                    lr=1e-5, lr_decay=False, reg=None, batch_size=1000,
                    use_bn=True, dropout_rate=0.4,
                    hidden_units=(128,64,32), loss_type="mse",)

In [ ]:
with tf.device('/GPU:0'):
  deepfm1.fit(
      train_data1to,
      neg_sampling=False,
      verbose=2,
      shuffle=True,
      eval_data=eval_data1to,
      metrics=["loss", "rmse"],
  )

Training start time: 2024-03-13 01:59:33


/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:47: UserWarning: `tf.layers.dropout` is deprecated and

total params: 66,448 | embedding params: 54,484 | network params: 11,964


train: 100%|██████████| 739/739 [00:10<00:00, 73.72it/s]


Epoch 1 elapsed: 10.029s
	 train_loss: 16.3805


...

	 eval rmse: 2.2129
	 eval rmse: 2.2129


train: 100%|██████████| 739/739 [00:09<00:00, 78.41it/s]


Epoch 12 elapsed: 9.435s
	 train_loss: 6.2226


eval_pointwise: 100%|██████████| 10/10 [00:00<00:00, 42.63it/s]


	 eval rmse: 2.0374
	 eval rmse: 2.0374


In [ ]:
test_data1.isna().sum()

user     0
item     0
label    0
dtype: int64

# **RESULTS: WITHOUT EMBEDDINGS**

In [ ]:
result_reviews1 = evaluate(model=deepfm1, data=test_data1to, neg_sampling='True', metrics=["loss", "rmse", "mae", "r2"])
print("evaluate_result: ", result_reviews1)

eval_pointwise: 100%|██████████| 23/23 [00:00<00:00, 63.65it/s]


evaluate_result:  {'loss': 1.6333135, 'rmse': 1.6333135, 'mae': 1.513509, 'r2': -9.670852748519001}


In [ ]:
result_reviews1 = evaluate(model=deepfm1, data=test_data1to, neg_sampling='False', metrics=["loss", "rmse", "mae", "r2"])
print("evaluate_result: ", result_reviews1)

eval_pointwise: 100%|██████████| 25/25 [00:00<00:00, 352.50it/s]

evaluate_result:  {'loss': 1.8094004, 'rmse': 1.8094004, 'mae': 1.703514, 'r2': -12.095718555428226}


In [ ]:
train_data1.head()

,user,item,label
0,1,1193,5
1,1,661,3
2,1,914,3
4,1,2355,5
6,1,1287,5


In [ ]:
test_data1.head()

,user,item,label
43,1,2692,4
44,1,260,4
45,1,1028,5
46,1,1029,5
47,1,1207,4


In [ ]:
train_data1.tail()


,user,item,label
1000139,6040,1392,5
1000140,6040,1394,3
1000141,6040,866,4
1000142,6040,2348,2
1000143,6040,2349,4


In [ ]:
deepfm1.predict(user=6040, item=1376)

2.2015058994293213

In [ ]:
predictions = []

for i in range(len(test_data1)):
  rate = deepfm1.predict(user = test_data1.iloc[i]['user'], item = test_data1.iloc[i]['item'])
  predictions.append(rate)


In [ ]:
train_data1.value_counts('label')

label
4    279127
3    210925
5    177747
2     86985
1     45409
dtype: int64

In [ ]:
predictions

[2.5067481994628906,
 1.9348050355911255,
 2.8809404373168945,
 2.2174596786499023,
 2.13161301612854,
 2.3477325439453125,
 2.420086145401001,
 1.996245265007019,
 2.7048556804656982,
 2.4815852642059326,
 2.193035840988159,
 2.3947601318359375,
 1.9534660577774048,
 2.1297144889831543,
 2.537461519241333,
 1.938478946685791,
 2.1684041023254395,
 1.7813308238983154,
 2.257148504257202,
 1.9406087398529053,
 2.11755633354187,
 1.8199963569641113,
 1.7984318733215332,
 2.414579391479492,
 2.1585867404937744,
 2.009782314300537,
 1.8729009628295898,
 2.295259475708008,
 1.550896167755127,
 2.343769073486328,
 2.2112293243408203,
 1.7598755359649658,
 1.9993963241577148,
 2.226912021636963,
 2.3284008502960205,
 2.450897693634033,
 2.1137752532958984,
 1.6762619018554688,
 2.241755485534668,
 2.2295007705688477,
 2.1382460594177246,
 2.37941312789917,
 3.0148048400878906,
 2.6425037384033203,
 2.3018646240234375,
 2.797032356262207,
 2.8334407806396484,
 2.516511917114258,
 2.60115432739